In [3]:
import numpy as np
from scipy.optimize import minimize
from openptv_python.epi import img_coord
from openptv_python.calibration import Calibration
from openptv_python.parameters import MultimediaPar, ControlPar

In [ ]:
# Based on the C tests in liboptv/tests/check_orientation.c

from openptv_python.imgcoord import image_coordinates
from openptv_python.parameters import OrientPar, read_control_par
from openptv_python.tracking_frame_buf import TargetArray
from openptv_python.trafo import arr_metric_to_pixel


control_file_name = "tests/testing_folder/corresp/control.par"
# control = ControlPar(4)
control = read_control_par(control_file_name)

orient_par_file_name = "tests/testing_folder/corresp/orient.par"
orient_par = OrientPar().from_file(orient_par_file_name)

cal = Calibration().from_file(
    "tests/testing_folder/calibration/cam1.tif.ori",
    "tests/testing_folder/calibration/cam1.tif.addpar",
)
orig_cal = Calibration().from_file(
    "tests/testing_folder/calibration/cam1.tif.ori",
    "tests/testing_folder/calibration/cam1.tif.addpar",
)


def test_external_calibration(self):
    """External calibration using clicked points."""
    ref_pts = np.array(
        [
            [-40.0, -25.0, 8.0],
            [40.0, -15.0, 0.0],
            [40.0, 15.0, 0.0],
            [40.0, 0.0, 8.0],
        ]
    )

    # Fake the image points by back-projection
    targets = arr_metric_to_pixel(
        image_coordinates(ref_pts, cal, control.mm),
        control,
    )

    # Jigg the fake detections to give raw_orient some challenge.
    targets[:, 1] -= 0.1

    # assertTrue(external_calibration(cal, ref_pts, targets, control))
    np.testing.assert_array_almost_equal(
        cal.get_angles(), orig_cal.get_angles(), decimal=3
    )
    np.testing.assert_array_almost_equal(
        cal.get_pos(), orig_cal.get_pos(), decimal=3
    )

"""Full calibration using clicked points."""
ref_pts = np.array(
    [
        a.flatten()
        for a in np.meshgrid(np.r_[-60:-30:4j], np.r_[0:15:4j], np.r_[0:15:4j])
    ]
).T

# Fake the image points by back-projection
targets = arr_metric_to_pixel(
    image_coordinates(ref_pts, cal, control.get_multimedia_params()),
    control,
)

# Full calibration works with TargetArray objects, not NumPy.
target_array = TargetArray(len(targets))
for i, trgt in enumerate(target_array):
    trgt.set_pnr(i)
    trgt.set_pos(targets[i])

# Perturb the calibration object, then compore result to original.
cal.set_pos(cal.get_pos() + np.r_[15.0, -15.0, 15.0])
cal.set_angles(cal.get_angles() + np.r_[-0.5, 0.5, -0.5])

In [4]:


def predict_2d_points(cal: Calibration, mm: MultimediaPar, fix: np.ndarray):
    # Placeholder function to predict 2D points based on calibration parameters and 3D points
    # You should replace this function with the actual implementation based on your calibration model
    xp, yp = img_coord(fix, cal, mm)
    return np.r_[xp, yp]

def cost_function(cal: Calibration, cpar: ControlPar, fix: np.ndarray, pix: np.ndarray):
    # Calculate the difference between predicted and actual 2D points
    predicted_points = np.array([predict_2d_points(cal, cpar.mm, point) for point in fix])
    residuals = pix - predicted_points.flatten()
    
    # Return the sum of squared residuals
    return np.sum(residuals**2)

# Example usage
# Assuming Calibration class is defined with appropriate parameters
# cal = Calibration()
# fix = np.array([[x1, y1, z1], [x2, y2, z2], ...])  # 3D points
# pix = np.array([[u1, v1], [u2, v2], ...])  # 2D points

# Define the optimization function for Scipy
def optimize_function(cal: Calibration, fix: np.ndarray, pix: np.ndarray, cpar: ControlPar):
    # cal = cal_flat.reshape(cal.shape)  # Reshape the flattened calibration parameters
    return cost_function(cal, cpar, fix, pix)

# Flatten the initial calibration parameters
def flat_calibration(cal: Calibration):
    return [cal.ext_par.x0, cal.ext_par.y0, cal.ext_par.z0, 
            cal.ext_par.omega, cal.ext_par.phi, cal.ext_par.kappa,
            cal.int_par.xh, cal.int_par.yh, cal.int_par.cc, 
            cal.added_par.k1, cal.added_par.k2, cal.added_par.k3, 
            cal.added_par.p1, cal.added_par.p2,
            cal.added_par.scx, cal.added_par.she]
    
cal = Calibration()
cal.from_file("tests/testing_fodder/calibration/cam0.ori")
initial_cal_flat = flat_calibration(cal)


# Use Scipy's minimize function
result = minimize(optimize_function, inital_cal_flat, args=(cpar, fix, pix), method='L-BFGS-B')

# Reshape the result to obtain the optimized calibration parameters
optimized_cal_flat = result.x
optimized_cal = optimized_cal_flat.reshape(cal.shape)


NameError: name 'cal' is not defined